# 4.1 Análise e exploração de dados

1.Construa um gráfico que permita visualizar os níveis médios do poluente O3 nas diversas regiões de 
Portugal (NUTS Code). Indique a região com nível médio de O3 mais elevado. 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Carregar os dados
df = pd.read_csv("AIRPOL_data.csv", sep=';')

# Converter colunas para numérico (substituindo vírgulas por pontos)
numeric_columns = ['Air_Pollution_Average[ug/m3]', 'Value', 'Populated_Area[km2]', 'Affected_Population']
for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
        

# Filtrar dados de Portugal e O3
df_portugal_o3 = df[(df['Country'] == 'Portugal') & (df['Air_Pollutant'] == 'O3')]

# Verificar dados
if df_portugal_o3.empty:
    print("Não existem dados de O3 para Portugal.")
    exit()


df_portugal_o3['NUTS_Code'] = df_portugal_o3['NUTS_Code'].astype(str)

# Calcular média por região (NUTS_Code)
o3_means = df_portugal_o3.groupby('NUTS_Code')['Air_Pollution_Average[ug/m3]'].mean()

# Gráfico de barras
plt.figure(figsize=(10, 6))
o3_means.plot(kind='bar', color='skyblue')
plt.title('Níveis Médios de O3 por Região em Portugal (2022)')
plt.xlabel('Região (NUTS Code)')
plt.ylabel('Nível Médio de O3')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Identificar região com maior nível médio
max_region = o3_means.idxmax()
max_value = o3_means.max()
print(f"Região com maior nível médio de O3: {max_region} ({max_value:.4f})")

2.Usando boxplots, construa um gráfico que permita comparar as distribuições dos níveis médios do
poluente PM2.5 em Portugal, Espanha, Franca e Itália.

In [ ]:
import seaborn as sns

# Filtrar países e PM2.5
countries = ['Portugal', 'Spain', 'France', 'Italy']
df_pm25 = df[(df['Country'].isin(countries)) & (df['Air_Pollutant'] == 'PM2.5')]

print(f"Total de dados de PM2.5: {len(df_pm25)}\n")
for country in countries:
    country_data = df_pm25[df_pm25['Country'] == country]
    print(f"Dados de PM2.5 para {country}: {len(country_data)}")
    # Apresentar o maior, menor e média
    print(f"  Maior: {country_data['Air_Pollution_Average[ug/m3]'].max():.4f}")
    print(f"  Menor: {country_data['Air_Pollution_Average[ug/m3]'].min():.4f}")
    print(f"  Média: {country_data['Air_Pollution_Average[ug/m3]'].mean():.4f}")
    print(f"  Desvio Padrão: {country_data['Air_Pollution_Average[ug/m3]'].std():.4f}")
    print(f"  Mediana : {country_data['Air_Pollution_Average[ug/m3]'].median():.4f}\n")


# Verificar se existem dados para os países selecionados
if df_pm25.empty:
    print("Não existem dados de PM2.5 para os países selecionados.")
    exit()

# Criar o boxplot
plt.figure(figsize=(10, 6))  # Define o tamanho do gráfico
sns.boxplot(x='Country', y='Air_Pollution_Average[ug/m3]', data=df_pm25, palette='Set2')  # Gera o boxplot
plt.title('Distribuição dos Níveis Médios de PM2.5 por País (2022)')  # Título
plt.xlabel('País')  # Eixo X
plt.ylabel('Nível Médio de PM2.5')  # Eixo Y
plt.tight_layout()  # Ajusta o layout
plt.show()  # Exibe o gráfico

3.Usando  uma  representação  gráfica adequada,  compare  as  distribuições  do  número  de mortes prematuras (Value) em Portugal, Espanha, França e Itália.

In [ ]:

# Filtrar dados dos países e mortes prematuras
df_mortes = df[df['Country'].isin(countries)]

print(f"\nTotal de dados de mortes prematuras: {len(df_mortes)}\n")  

for country in countries:
    country_data = df_mortes[df_mortes['Country'] == country]
    print(f"Dados de mortes prematuras para {country}: {len(country_data)}")
    # Apresentar o maior, menor e média
    print(f"  Maior: {country_data['Value'].max():.4f}")
    print(f"  Menor: {country_data['Value'].min():.4f}")
    print(f"  Média: {country_data['Value'].mean():.4f}")
    print(f"  Desvio Padrão: {country_data['Value'].std():.4f}")
    print(f"  Mediana : {country_data['Value'].median():.4f}")
    print(f"  Total de mortes: {country_data['Value'].sum():.4f}")
    print("\n")


# Criar o gráfico de boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x='Country', y='Value', data=df_mortes, palette='Set3')
plt.title('Distribuição do Número de Mortes Prematuras por País')
plt.xlabel('País')
plt.ylabel('Número de Mortes Prematuras (Value)')
plt.yscale('log')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


4.Construa uma tabela que indique, os valores da média, quartis, desvio padrão, assimetria e curtose, do 
número de mortes prematuras (Value) associado a STROKE para cada um dos países: Espanha,  França, 
Itália e Grécia (apresente os resultados com 4 casas decimais)

In [ ]:
from scipy.stats import skew, kurtosis
from tabulate import tabulate

# Filtrar dados de mortes prematuras associadas a STROKE para os países especificados
df_stroke = df[(df['Country'].isin(countries)) & (df['Outcome'] == 'Stroke')]


# Função para calcular as estatísticas
def calc_stats(group):
    return pd.Series({
        'Média': group.mean(),
        'Q1': group.quantile(0.25),
        'Mediana': group.median(),
        'Q3': group.quantile(0.75),
        'Desvio Padrão': group.std(),
        'Assimetria': skew(group),
        'Curtose': kurtosis(group)
    })

# Calcular estatísticas por país
stats_table = df_stroke.groupby('Country')['Value'].apply(calc_stats)
# Arredondar para 4 casas decimais
stats_table = stats_table.round(4)

print(tabulate(stats_table.unstack().T, headers='keys', tablefmt='pretty', showindex=True))

